# Lib

In [50]:
%load_ext autoreload
%autoreload 2

import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)


import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
import glob

if sys.platform=='win32':
    sys.path.insert(0,".\..\src")
        
elif sys.platform=='linux':
    sys.path.insert(0,"./../src")

from utils import utils_ml, parameters

from ipywidgets import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
if sys.platform=='win32':
    !dir .\
        
elif sys.platform=='linux':
    !ls ./

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks

18/11/2022  19:16    <DIR>          .
18/11/2022  16:18    <DIR>          ..
10/09/2022  23:28    <DIR>          .ipynb_checkpoints
18/11/2022  16:18         1.047.962 1.EA.ipynb
18/11/2022  20:25            67.359 2.Feature_engineering.ipynb
18/11/2022  20:26            20.594 3.Stability_var.ipynb
18/11/2022  18:02           182.210 4.Modeling.ipynb
18/11/2022  20:25    <DIR>          catboost_info
               4 arquivo(s)      1.318.125 bytes
               4 pasta(s)   341.458.522.112 bytes dispon�veis


# Reading

In [3]:
## para saber en que directório uno se encuentra
#
if sys.platform=='win32':
    !echo %cd%
        
elif sys.platform=='linux':
    !pwd

c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks


In [4]:
df = pd.read_parquet(".\\..\data\dados_transformed.parquet")
df['data'] = pd.to_datetime(df['data'])

In [5]:
features_bins_woe = utils_ml.list_subset_words( df.columns.tolist(), ["bin"])
## Excluding variable 'score' (score of Meli model) and 'o_obj' (with 72.5% of missing values))
features_bins_woe = utils_ml.exclude_words(features_bins_woe, ["score","o_obj"])
features_bins_woe

['n_boolean_bins',
 'p_boolean_bins',
 'categoria_produto_bins',
 'pais_bins',
 'a_int_bins',
 'b_float_bins',
 'c_float_bins',
 'd_float_bins',
 'e_float_bins',
 'f_float_bins',
 'h_float_bins',
 'k_float_bins',
 'l_float_bins',
 'm_float_bins',
 'monto_bins']

# Model

## Logistic regression

In [6]:
df_2_model = df[features_bins_woe + ["fraude"]]

### Dumming features

In [7]:
cat_vars = features_bins_woe
df_2_model_dum = df_2_model.copy()

for var in cat_vars:

    cat_list='var'+'_'+var
    cat_list = pd.get_dummies( df_2_model_dum[var] , prefix=var)
    data_dum = df_2_model_dum.join(cat_list)
    df_2_model_dum = data_dum


data_vars = df_2_model_dum.columns.tolist()
to_keep = [i for i in data_vars if i not in cat_vars]

df_2_model_dum = df_2_model_dum[to_keep]


### Train and test samples

In [8]:
from sklearn.model_selection import train_test_split

X = df_2_model_dum.drop(['fraude'], axis=1)
y = df_2_model_dum['fraude']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=0, stratify=y )

In [9]:
y.value_counts(normalize=True)[0]/y.value_counts(normalize=True)[1]

18.999999999999996

### Trainning

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# lr = LogisticRegression(random_state=0,solver='newton-cg', fit_intercept=True, penalty='l2' )

# #Setting the range for class weights
# weights = np.linspace(0.0,0.99,200)

# #Creating a dictionary grid for grid search
# param_grid = {'class_weight': [{0:x, 1:1.0-x} for x in weights]}

# #Fitting grid search to the train data with 5 folds
# gridsearch = GridSearchCV(estimator= lr, 
#                           param_grid= param_grid,
#                           cv=StratifiedKFold(), 
#                           n_jobs=-1, 
#                           scoring='recall', 
#                           verbose=2).fit(X_train, y_train)

## Ploting the score for different values of weight
# sns.set_style('whitegrid')
# plt.figure(figsize=(12,8))
# weigh_data = pd.DataFrame({ 'score': gridsearch.cv_results_['mean_test_score'], 'weight': (1-weights)})
# sns.lineplot(weigh_data,  x='weight', y='score')
# plt.xlabel('Weight for class 1')
# plt.ylabel('F1 score')
# plt.xticks([round(i/10,1) for i in range(0,11,1)])
# plt.title('Scoring for different class weights', fontsize=24)

In [11]:
# LogisticRegression??

In [12]:
clf = LogisticRegression( random_state=0, solver='saga',l1_ratio = 0.1, fit_intercept=True, penalty='elasticnet', class_weight = "balanced" )

model = clf.fit(X_train, y_train )

c:\Users\caanp\miniconda3\envs\venv_ml\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [13]:
df_2_model['probabilty'] = model.predict_proba(X)[:,1]

C:\Users\caanp\AppData\Local\Temp\ipykernel_7748\2283094811.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_model['probabilty'] = model.predict_proba(X)[:,1]


In [14]:
colunas_orig = utils_ml.exclude_words( df.columns.tolist(), ["bin","cat","WoE"] ) + ['categoria_produto','produto']
# colunas_orig

In [15]:
df_ivs_ls = []

for variable in colunas_orig:

    label = "fraude"
    dtype_ = 'categorical' if df[variable].dtype=='O' or df[variable].dtype=='category' else 'numerical'

    df_iv = utils_ml.WOE_IV(df = df , variable = variable, dtype_ = dtype_ , label = label, monotonic= True)[[ 'variable' , 'IV_var', 'KS']].drop_duplicates()

    df_ivs_ls.append(df_iv)

pd.concat(df_ivs_ls).sort_values("IV_var", ascending=False)

,variable,IV_var,KS
0,fecha,12.113640,99.544561
0,produto,6.369097,88.306667
0,produto,6.369097,88.306667
0,hora,4.346361,84.314386
0,categoria_produto,1.641776,39.766316
0,o_obj,1.094238,44.994386
0,score,1.021894,43.152982
0,f_float,0.447805,27.785965
0,l_float,0.439939,25.305965
0,n_boolean,0.361450,22.844211


In [16]:
coef_df = pd.DataFrame( {"Cols": model.feature_names_in_,  "coef": model.coef_[0] })
coef_df.sort_values("coef", ascending=False)

,Cols,coef
12,categoria_produto_bins_8,2.970717
15,categoria_produto_bins_11,2.825391
78,m_float_bins_0,2.277403
14,categoria_produto_bins_10,2.252578
71,l_float_bins_6,2.160489
67,l_float_bins_2,2.130371
13,categoria_produto_bins_9,1.871387
40,e_float_bins_1,1.671934
63,k_float_bins_2,1.525753
25,b_float_bins_3,1.512607


### comparing with Meli model

> **modelo MELI**

In [17]:
probability = "score"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df, probability, label, quantil)

,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,15000,"[0, 8]",417,14583,7500,142500,0,0,5.560000,10.233684,5.560000,10.233684,42.385965,1.000000,0.050000,0.000000,0.095238,0.208333
1,15000,"[8, 18]",692,14308,7083,127917,14583,417,9.226667,10.040702,14.786667,20.274386,42.385965,0.944400,0.052467,0.102337,0.099411,0.214636
2,15000,"[18, 28]",494,14506,6391,113609,28891,1109,6.586667,10.179649,21.373333,30.454035,42.385965,0.852133,0.053258,0.202744,0.100251,0.213033
3,15000,"[28, 38]",46,14954,5897,99103,43397,1603,0.613333,10.494035,21.986667,40.948070,42.385965,0.786267,0.056162,0.304540,0.104836,0.218407
4,15000,"[38, 48]",88,14912,5851,84149,58351,1649,1.173333,10.464561,23.160000,51.412632,42.385965,0.780133,0.065011,0.409481,0.120021,0.243792
5,15000,"[48, 58]",196,14804,5763,69237,73263,1737,2.613333,10.388772,25.773333,61.801404,42.385965,0.768400,0.076840,0.514126,0.139709,0.274429
6,15000,"[58, 68]",297,14703,5567,54433,88067,1933,3.960000,10.317895,29.733333,72.119298,42.385965,0.742267,0.092783,0.618014,0.164948,0.309278
7,15000,"[68, 78]",780,14220,5270,39730,102770,2230,10.400000,9.978947,40.133333,82.098246,42.385965,0.702667,0.117111,0.721193,0.200762,0.351333
8,15000,"[78, 88]",1283,13717,4490,25510,116990,3010,17.106667,9.625965,57.240000,91.724211,42.385965,0.598667,0.149667,0.820982,0.239467,0.374167


> **Logistic Regression**

In [18]:
probability = "probabilty"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df_2_model, probability, label, quantil)

,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,15000,"[3.3982149677952545e-10, 9.78692699804565e-06]",12,14988,7500,142500,0,0,0.160000,10.517895,0.160000,10.517895,39.087719,1.000000,0.050000,0.000000,0.095238,0.208333
1,15000,"[9.787779421938884e-06, 0.00015692048082579122]",31,14969,7488,127512,14988,12,0.413333,10.504561,0.573333,21.022456,39.087719,0.998400,0.055467,0.105179,0.105095,0.226909
2,15000,"[0.00015694696537023412, 0.001696293573806099]",152,14848,7457,112543,29957,43,2.026667,10.419649,2.600000,31.442105,39.087719,0.994267,0.062142,0.210225,0.116973,0.248567
3,15000,"[0.0016963137449009945, 0.009097938296410491]",345,14655,7305,97695,44805,195,4.600000,10.284211,7.200000,41.726316,39.087719,0.974000,0.069571,0.314421,0.129867,0.270556
4,15000,"[0.009098090471595478, 0.034898628428843746]",523,14477,6960,83040,59460,540,6.973333,10.159298,14.173333,51.885614,39.087719,0.928000,0.077333,0.417263,0.142769,0.290000
5,15000,"[0.03489877154565682, 0.1089882260954484]",652,14348,6437,68563,73937,1063,8.693333,10.068772,22.866667,61.954386,39.087719,0.858267,0.085827,0.518856,0.156048,0.306524
6,15000,"[0.10899253366995107, 0.2941455644701354]",853,14147,5785,54215,88285,1715,11.373333,9.927719,34.240000,71.882105,39.087719,0.771333,0.096417,0.619544,0.171407,0.321389
7,15000,"[0.29414787142504567, 0.6193969171408913]",1092,13908,4932,40068,102432,2568,14.560000,9.760000,48.800000,81.642105,39.087719,0.657600,0.109600,0.718821,0.187886,0.328800
8,15000,"[0.6194005424414122, 0.9071735083656679]",1547,13453,3840,26160,116340,3660,20.626667,9.440702,69.426667,91.082807,39.087719,0.512000,0.128000,0.816421,0.204800,0.320000


>> **Logistic Regression in test sample**

In [19]:
df_test = pd.DataFrame()
df_test['fraude'] = y_test
df_test['probabilty'] = model.predict_proba(X_test)[:,1]

probability = "probabilty"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df_test, probability, label, quantil)

,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,4500,"[4.0007107648878425e-10, 1.0485862983142917e-05]",8,4492,2250,42750,0,0,0.355556,10.507602,0.355556,10.507602,37.988304,1.000000,0.050000,0.000000,0.095238,0.208333
1,4500,"[1.048673423567611e-05, 0.00017704396727571096]",12,4488,2242,38258,4492,8,0.533333,10.498246,0.888889,21.005848,37.988304,0.996444,0.055358,0.105076,0.104889,0.226465
2,4500,"[0.00017725371231411618, 0.001741433817667927]",45,4455,2230,33770,8980,20,2.000000,10.421053,2.888889,31.426901,37.988304,0.991111,0.061944,0.210058,0.116601,0.247778
3,4500,"[0.0017420898931864847, 0.009144915586801717]",105,4395,2185,29315,13435,65,4.666667,10.280702,7.555556,41.707602,37.988304,0.971111,0.069365,0.314269,0.129481,0.269753
4,4500,"[0.00914518116974714, 0.035380292019279516]",166,4334,2080,24920,17830,170,7.377778,10.138012,14.933333,51.845614,37.988304,0.924444,0.077037,0.417076,0.142222,0.288889
5,4500,"[0.03538947354876463, 0.1102054083618461]",202,4298,1914,20586,22164,336,8.977778,10.053801,23.911111,61.899415,37.988304,0.850667,0.085067,0.518456,0.154667,0.303810
6,4500,"[0.11023264945593543, 0.2990615512851331]",262,4238,1712,16288,26462,538,11.644444,9.913450,35.555556,71.812865,37.988304,0.760889,0.095111,0.618994,0.169086,0.317037
7,4500,"[0.2990709166389137, 0.6260726145565355]",347,4153,1450,12050,30700,800,15.422222,9.714620,50.977778,81.527485,37.988304,0.644444,0.107407,0.718129,0.184127,0.322222
8,4500,"[0.6261585595006027, 0.9079630548358851]",441,4059,1103,7897,34853,1147,19.600000,9.494737,70.577778,91.022222,37.988304,0.490222,0.122556,0.815275,0.196089,0.306389


## Catboost

In [20]:
# !pip install catboost

In [21]:
import catboost as cb

### Train and test samples

In [22]:
cat_features = [x for x in utils_ml.list_subset_words( df.columns.tolist(), ['cat']) if "WoE" not in x and "bin" not in x and x!='categoria_produto']

df_2_model_cat = df[cat_features + ["fraude"]]


X = df_2_model_cat.drop(['fraude'], axis=1)
y = df_2_model_cat['fraude']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=0, stratify=y )

### Trainning

In [23]:
model_cat_def = cb.CatBoostClassifier(
                           l2_leaf_reg = 3.0,
                           scale_pos_weight = 19,
                           verbose=False,
                           thread_count = 8,
                           random_seed = 0
                          )


modelo_cat = model_cat_def.fit(X_test , y_test, cat_features =  cat_features  )

In [24]:
coef = pd.DataFrame( {"features":modelo_cat.feature_names_,"coef":modelo_cat.feature_importances_})
coef = coef.sort_values("coef", ascending=False).reset_index(drop=True).query("coef>0")
coef

,features,coef
0,categoria_produto_cat,35.722931
1,o_obj_cat,10.118526
2,l_float_cat,7.436246
3,monto_cat,7.341126
4,score_cat,6.984067
5,f_float_cat,6.945596
6,b_float_cat,4.165394
7,d_float_cat,4.118669
8,m_float_cat,3.693161
9,h_float_cat,3.123742


### comparing with Meli model

> **modelo MELI**

In [25]:
probability = "score"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df, probability, label, quantil)

,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,15000,"[0, 8]",417,14583,7500,142500,0,0,5.560000,10.233684,5.560000,10.233684,42.385965,1.000000,0.050000,0.000000,0.095238,0.208333
1,15000,"[8, 18]",692,14308,7083,127917,14583,417,9.226667,10.040702,14.786667,20.274386,42.385965,0.944400,0.052467,0.102337,0.099411,0.214636
2,15000,"[18, 28]",494,14506,6391,113609,28891,1109,6.586667,10.179649,21.373333,30.454035,42.385965,0.852133,0.053258,0.202744,0.100251,0.213033
3,15000,"[28, 38]",46,14954,5897,99103,43397,1603,0.613333,10.494035,21.986667,40.948070,42.385965,0.786267,0.056162,0.304540,0.104836,0.218407
4,15000,"[38, 48]",88,14912,5851,84149,58351,1649,1.173333,10.464561,23.160000,51.412632,42.385965,0.780133,0.065011,0.409481,0.120021,0.243792
5,15000,"[48, 58]",196,14804,5763,69237,73263,1737,2.613333,10.388772,25.773333,61.801404,42.385965,0.768400,0.076840,0.514126,0.139709,0.274429
6,15000,"[58, 68]",297,14703,5567,54433,88067,1933,3.960000,10.317895,29.733333,72.119298,42.385965,0.742267,0.092783,0.618014,0.164948,0.309278
7,15000,"[68, 78]",780,14220,5270,39730,102770,2230,10.400000,9.978947,40.133333,82.098246,42.385965,0.702667,0.117111,0.721193,0.200762,0.351333
8,15000,"[78, 88]",1283,13717,4490,25510,116990,3010,17.106667,9.625965,57.240000,91.724211,42.385965,0.598667,0.149667,0.820982,0.239467,0.374167


> **Catboost model**

In [26]:
df_2_model_cat['probabilty'] = modelo_cat.predict_proba(X)[:,1]

probability = "probabilty"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df_2_model_cat, probability, label, quantil)

C:\Users\caanp\AppData\Local\Temp\ipykernel_7748\240659597.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_model_cat['probabilty'] = modelo_cat.predict_proba(X)[:,1]


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,15000,"[7.71438019804436e-05, 0.003229909900620665]",2,14998,7500,142500,0,0,0.026667,10.524912,0.026667,10.524912,66.161404,1.000000,0.050000,0.000000,0.095238,0.208333
1,15000,"[0.003230323923035018, 0.019109729315543125]",2,14998,7498,127502,14998,2,0.026667,10.524912,0.053333,21.049825,66.161404,0.999733,0.055541,0.105249,0.105235,0.227212
2,15000,"[0.01911159540564689, 0.06807829472070138]",44,14956,7496,112504,29996,4,0.586667,10.495439,0.640000,31.545263,66.161404,0.999467,0.062467,0.210498,0.117584,0.249867
3,15000,"[0.06807848878797317, 0.10462216311880024]",71,14929,7452,97548,44952,48,0.946667,10.476491,1.586667,42.021754,66.161404,0.993600,0.070971,0.315453,0.132480,0.276000
4,15000,"[0.10462396059257056, 0.14279666111778228]",94,14906,7381,82619,59881,119,1.253333,10.460351,2.840000,52.482105,66.161404,0.984133,0.082011,0.420218,0.151405,0.307542
5,15000,"[0.1427996880596146, 0.19426974989348558]",160,14840,7287,67713,74787,213,2.133333,10.414035,4.973333,62.896140,66.161404,0.971600,0.097160,0.524821,0.176655,0.347000
6,15000,"[0.19427341732585965, 0.2878895694010516]",301,14699,7127,52873,89627,373,4.013333,10.315088,8.986667,73.211228,66.161404,0.950267,0.118783,0.628961,0.211170,0.395944
7,15000,"[0.28789241708578867, 0.44679991824370036]",612,14388,6826,38174,104326,674,8.160000,10.096842,17.146667,83.308070,66.161404,0.910133,0.151689,0.732112,0.260038,0.455067
8,15000,"[0.44681204040695166, 0.6582698615124198]",1174,13826,6214,23786,118714,1286,15.653333,9.702456,32.800000,93.010526,66.161404,0.828533,0.207133,0.833081,0.331413,0.517833


>> **Catboost in test sample**

In [27]:
df_test = pd.DataFrame()
df_test['fraude'] = y_test
df_test['probabilty'] = modelo_cat.predict_proba(X_test)[:,1]

probability = "probabilty"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df_test, probability, label, quantil)

,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,4500,"[7.71438019804436e-05, 0.0033114219263442315]",0,4500,2250,42750,0,0,0.000000,10.526316,0.000000,10.526316,73.263158,1.000000,0.050000,0.000000,0.095238,0.208333
1,4500,"[0.003311967130632306, 0.02132902520975196]",0,4500,2250,38250,4500,0,0.000000,10.526316,0.000000,21.052632,73.263158,1.000000,0.055556,0.105263,0.105263,0.227273
2,4500,"[0.02136735142666924, 0.0691711294313374]",0,4500,2250,33750,9000,0,0.000000,10.526316,0.000000,31.578947,73.263158,1.000000,0.062500,0.210526,0.117647,0.250000
3,4500,"[0.0691719215307923, 0.10587070575688665]",4,4496,2250,29250,13500,0,0.177778,10.516959,0.177778,42.095906,73.263158,1.000000,0.071429,0.315789,0.133333,0.277778
4,4500,"[0.10588341680994681, 0.14317445610432633]",2,4498,2246,24754,17996,4,0.088889,10.521637,0.266667,52.617544,73.263158,0.998222,0.083185,0.420959,0.153573,0.311944
5,4500,"[0.14317451087846314, 0.19469737698094183]",21,4479,2244,20256,22494,6,0.933333,10.477193,1.200000,63.094737,73.263158,0.997333,0.099733,0.526175,0.181333,0.356190
6,4500,"[0.19474452909832513, 0.29135547696908115]",66,4434,2223,15777,26973,27,2.933333,10.371930,4.133333,73.466667,73.263158,0.988000,0.123500,0.630947,0.219556,0.411667
7,4500,"[0.29138207776819824, 0.4481734687705463]",141,4359,2157,11343,31407,93,6.266667,10.196491,10.400000,83.663158,73.263158,0.958667,0.159778,0.734667,0.273905,0.479333
8,4500,"[0.4482869866001156, 0.6582260022999138]",346,4154,2016,6984,35766,234,15.377778,9.716959,25.777778,93.380117,73.263158,0.896000,0.224000,0.836632,0.358400,0.560000


## Using period in evaluation

> **modelo MELI**

Apresenta uma relativa estabilidade quando se olha os score para as semanas

In [28]:
gb = df.groupby(pd.Grouper(key='data', freq="W"))

dff_with_week = []

for week, df_week in gb:

    df_week['week'] = week

    dff_with_week.append(df_week)


dff_week = pd.concat(dff_with_week)

C:\Users\caanp\AppData\Local\Temp\ipykernel_7748\26454910.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_week['week'] = week
C:\Users\caanp\AppData\Local\Temp\ipykernel_7748\26454910.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_week['week'] = week
C:\Users\caanp\AppData\Local\Temp\ipykernel_7748\26454910.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [124]:
probability = "score"
label="fraude"
quantil=10

@interact

def meli_model(week = weeks):

    df_marco = dff_week[dff_week['week']==week]

    display(utils_ml.metric_evaluation(df_marco, probability, label, quantil))

interactive(children=(Dropdown(description='week', options=(numpy.datetime64('2020-03-08T00:00:00.000000000'),…

> **Catboost**

Apresenta uma relativa estabilidade no tempo quando se olha para as semanas

In [68]:
probability = "probabilty"
label="fraude"
quantil=10

@interact

def cat_model(week = weeks):

    df_marco_cat = dff_week[dff_week['week']==week][cat_features+['fraude']]

    df_marco_cat['probabilty'] = modelo_cat.predict_proba( df_marco_cat )[:,1]

    display(utils_ml.metric_evaluation(df_marco_cat, probability, label, quantil))

interactive(children=(Dropdown(description='week', options=(numpy.datetime64('2020-03-08T00:00:00.000000000'),…

> **Logistic Regression**

Apresenta instabilidade quando se escore por semana e se olha para os decis

In [120]:
def transform(df , cols = features_bins_woe + ["fraude"]):

    colus_regression = ['fraude', 'n_boolean_bins_0', 'n_boolean_bins_1', 'p_boolean_bins_0','p_boolean_bins_1', 'categoria_produto_bins_0', 'categoria_produto_bins_1', 'categoria_produto_bins_2', 'categoria_produto_bins_3', 'categoria_produto_bins_4', 'categoria_produto_bins_5', 'categoria_produto_bins_6', 'categoria_produto_bins_7', 'categoria_produto_bins_8', 'categoria_produto_bins_9', 'categoria_produto_bins_10', 'categoria_produto_bins_11', 'pais_bins_-1', 'pais_bins_0', 'pais_bins_1', 'a_int_bins_0', 'a_int_bins_1', 'b_float_bins_-1', 'b_float_bins_0', 'b_float_bins_1', 'b_float_bins_2', 'b_float_bins_3', 'b_float_bins_4', 'b_float_bins_5', 'b_float_bins_6', 'c_float_bins_-1', 'c_float_bins_0', 'c_float_bins_1', 'd_float_bins_-1', 'd_float_bins_0', 'd_float_bins_1', 'd_float_bins_2', 'd_float_bins_3', 'd_float_bins_4', 'd_float_bins_5', 'e_float_bins_0', 'e_float_bins_1', 'e_float_bins_2', 'f_float_bins_-1', 'f_float_bins_0', 'f_float_bins_1', 'f_float_bins_2', 'f_float_bins_3', 'f_float_bins_4', 'f_float_bins_5', 'f_float_bins_6', 'f_float_bins_7', 'f_float_bins_8', 'f_float_bins_9', 'f_float_bins_10', 'h_float_bins_0', 'h_float_bins_1', 'h_float_bins_2', 'h_float_bins_3', 'h_float_bins_4', 'h_float_bins_5', 'h_float_bins_6', 'k_float_bins_0', 'k_float_bins_1', 'k_float_bins_2', 'l_float_bins_-1', 'l_float_bins_0', 'l_float_bins_1', 'l_float_bins_2', 'l_float_bins_3', 'l_float_bins_4', 'l_float_bins_5', 'l_float_bins_6', 'l_float_bins_7', 'l_float_bins_8', 'l_float_bins_9', 'l_float_bins_10', 'l_float_bins_11', 'm_float_bins_-1', 'm_float_bins_0', 'm_float_bins_1', 'm_float_bins_2', 'm_float_bins_3', 'm_float_bins_4', 'm_float_bins_5', 'm_float_bins_6', 'm_float_bins_7', 'm_float_bins_8', 'm_float_bins_9', 'monto_bins_0', 'monto_bins_1', 'monto_bins_2', 'monto_bins_3', 'monto_bins_4', 'monto_bins_5']

    df_marco_lg = df[features_bins_woe + ["fraude"]]
    cat_vars = features_bins_woe
    df_marco_model_dum = df_marco_lg.copy()

    for var in cat_vars:

        cat_list='var'+'_'+var
        cat_list = pd.get_dummies( df_marco_model_dum[var] , prefix=var)
        data_dum = df_marco_model_dum.join(cat_list)
        df_marco_model_dum = data_dum


    data_vars = df_marco_model_dum.columns.tolist()
    to_keep = [i for i in data_vars if i not in cat_vars]

    df_marco_model_dum = df_marco_model_dum[to_keep]

    dummies_faltantes = list(   set(colus_regression) - set(df_marco_model_dum.columns.to_list()) )

    if len(dummies_faltantes)>0:

        for dummi_falt in dummies_faltantes:

            df_marco_model_dum[dummi_falt] = 0

    return df_marco_model_dum

In [123]:
probability = "probabilty"
label="fraude"
quantil=10

@interact

def cat_model(week = weeks):

    df = dff_week[dff_week['week']==week]

    df_marco_model_dum = transform(df , cols = features_bins_woe + ["fraude"])

    ######### Fitting ###########################
    df_marco_model_dum['probabilty'] = model.predict_proba( df_marco_model_dum.drop("fraude", axis=1) )[:,1]



    display(utils_ml.metric_evaluation(df_marco_model_dum, probability, label, quantil))

interactive(children=(Dropdown(description='week', options=(numpy.datetime64('2020-03-08T00:00:00.000000000'),…

In [38]:
!lsb_release -a

'lsb_release' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


# Requirements

In [110]:
path_root = parameters.root
!pip list --format=freeze > $path_root/requirements.txt

5697.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
